<a href="https://colab.research.google.com/github/AmiraAshraf2/NLP/blob/main/NLP_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import numpy as np
import math
from sklearn.preprocessing import normalize

In [3]:
text_generation_pipeline = pipeline("text-generation")
documentA = text_generation_pipeline("the sun is yellow")[0]["generated_text"]
documentB = text_generation_pipeline("the moon is white")[0]["generated_text"]

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [4]:
documentA

"the sun is yellow and it's burning through a hole in it)\n\nA white woman who looks in an airtight glass case at the far end of her family. She wears two red sneakers, a pair of shoes made from rubber, and"

In [5]:
documentB

'the moon is white in the night, so why bother with that?"\n\n"Then you\'re gonna be a fool," Anna said, her voice filled with frustration. This was probably the closest she had come to being annoyed at the other woman.'

In [6]:
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())

    tokens = nltk.word_tokenize(text)


    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    return filtered_tokens

In [7]:
bagofwordsA = preprocess_text(documentA)
bagofwordsB = preprocess_text(documentB)

In [8]:
bagofwordsA

['sun',
 'yellow',
 'burning',
 'hole',
 'white',
 'woman',
 'looks',
 'airtight',
 'glass',
 'case',
 'far',
 'end',
 'family',
 'wears',
 'two',
 'red',
 'sneakers',
 'pair',
 'shoes',
 'made',
 'rubber']

In [9]:
bagofwordsB

['moon',
 'white',
 'night',
 'bother',
 'youre',
 'gon',
 'na',
 'fool',
 'anna',
 'said',
 'voice',
 'filled',
 'frustration',
 'probably',
 'closest',
 'come',
 'annoyed',
 'woman']

In [10]:
uniquewords = set(bagofwordsA).union(set(bagofwordsB))

In [11]:
uniquewords

{'airtight',
 'anna',
 'annoyed',
 'bother',
 'burning',
 'case',
 'closest',
 'come',
 'end',
 'family',
 'far',
 'filled',
 'fool',
 'frustration',
 'glass',
 'gon',
 'hole',
 'looks',
 'made',
 'moon',
 'na',
 'night',
 'pair',
 'probably',
 'red',
 'rubber',
 'said',
 'shoes',
 'sneakers',
 'sun',
 'two',
 'voice',
 'wears',
 'white',
 'woman',
 'yellow',
 'youre'}

In [12]:
numofwordsA = dict.fromkeys (uniquewords, 0)
for word in bagofwordsA:
  numofwordsA [word] += 1

numOfWordsB = dict.fromkeys (uniquewords, 0)

for word in bagofwordsB:
   numOfWordsB [word] += 1

In [13]:
numofwordsA

{'bother': 0,
 'gon': 0,
 'annoyed': 0,
 'case': 1,
 'family': 1,
 'pair': 1,
 'wears': 1,
 'airtight': 1,
 'burning': 1,
 'two': 1,
 'anna': 0,
 'yellow': 1,
 'made': 1,
 'moon': 0,
 'filled': 0,
 'fool': 0,
 'na': 0,
 'red': 1,
 'shoes': 1,
 'hole': 1,
 'glass': 1,
 'youre': 0,
 'said': 0,
 'woman': 1,
 'voice': 0,
 'closest': 0,
 'white': 1,
 'rubber': 1,
 'looks': 1,
 'come': 0,
 'probably': 0,
 'night': 0,
 'far': 1,
 'end': 1,
 'sneakers': 1,
 'frustration': 0,
 'sun': 1}

In [14]:
numOfWordsB

{'bother': 1,
 'gon': 1,
 'annoyed': 1,
 'case': 0,
 'family': 0,
 'pair': 0,
 'wears': 0,
 'airtight': 0,
 'burning': 0,
 'two': 0,
 'anna': 1,
 'yellow': 0,
 'made': 0,
 'moon': 1,
 'filled': 1,
 'fool': 1,
 'na': 1,
 'red': 0,
 'shoes': 0,
 'hole': 0,
 'glass': 0,
 'youre': 1,
 'said': 1,
 'woman': 1,
 'voice': 1,
 'closest': 1,
 'white': 1,
 'rubber': 0,
 'looks': 0,
 'come': 1,
 'probably': 1,
 'night': 1,
 'far': 0,
 'end': 0,
 'sneakers': 0,
 'frustration': 1,
 'sun': 0}

In [15]:
def computeTF (wordDict, bagofwords):

  tfDict = {}

  bagofwordsCount = len(bagofwords)

  for word, count in wordDict.items(): tfDict[word] = count / float (bagofwordsCount)
  return tfDict

In [16]:
tfA = computeTF (numofwordsA, bagofwordsA)

tfB = computeTF (numOfWordsB, bagofwordsB)

In [17]:
tfA

{'bother': 0.0,
 'gon': 0.0,
 'annoyed': 0.0,
 'case': 0.047619047619047616,
 'family': 0.047619047619047616,
 'pair': 0.047619047619047616,
 'wears': 0.047619047619047616,
 'airtight': 0.047619047619047616,
 'burning': 0.047619047619047616,
 'two': 0.047619047619047616,
 'anna': 0.0,
 'yellow': 0.047619047619047616,
 'made': 0.047619047619047616,
 'moon': 0.0,
 'filled': 0.0,
 'fool': 0.0,
 'na': 0.0,
 'red': 0.047619047619047616,
 'shoes': 0.047619047619047616,
 'hole': 0.047619047619047616,
 'glass': 0.047619047619047616,
 'youre': 0.0,
 'said': 0.0,
 'woman': 0.047619047619047616,
 'voice': 0.0,
 'closest': 0.0,
 'white': 0.047619047619047616,
 'rubber': 0.047619047619047616,
 'looks': 0.047619047619047616,
 'come': 0.0,
 'probably': 0.0,
 'night': 0.0,
 'far': 0.047619047619047616,
 'end': 0.047619047619047616,
 'sneakers': 0.047619047619047616,
 'frustration': 0.0,
 'sun': 0.047619047619047616}

In [18]:
tfB

{'bother': 0.05555555555555555,
 'gon': 0.05555555555555555,
 'annoyed': 0.05555555555555555,
 'case': 0.0,
 'family': 0.0,
 'pair': 0.0,
 'wears': 0.0,
 'airtight': 0.0,
 'burning': 0.0,
 'two': 0.0,
 'anna': 0.05555555555555555,
 'yellow': 0.0,
 'made': 0.0,
 'moon': 0.05555555555555555,
 'filled': 0.05555555555555555,
 'fool': 0.05555555555555555,
 'na': 0.05555555555555555,
 'red': 0.0,
 'shoes': 0.0,
 'hole': 0.0,
 'glass': 0.0,
 'youre': 0.05555555555555555,
 'said': 0.05555555555555555,
 'woman': 0.05555555555555555,
 'voice': 0.05555555555555555,
 'closest': 0.05555555555555555,
 'white': 0.05555555555555555,
 'rubber': 0.0,
 'looks': 0.0,
 'come': 0.05555555555555555,
 'probably': 0.05555555555555555,
 'night': 0.05555555555555555,
 'far': 0.0,
 'end': 0.0,
 'sneakers': 0.0,
 'frustration': 0.05555555555555555,
 'sun': 0.0}

In [19]:
def computeIDF (documents):



  N = len(documents)

  idfDict = dict.fromkeys(documents[0].keys(), 0)

  for document in documents:

    for word, val in document.items():

      if val > 0:

        idfDict[word] += 1
  for word, val in idfDict.items():

    idfDict[word] = math.log(N+1 / float(val)+1)+1

  return idfDict

In [20]:
idfs = computeIDF([numofwordsA, numOfWordsB])

In [21]:
idfs

{'bother': 2.386294361119891,
 'gon': 2.386294361119891,
 'annoyed': 2.386294361119891,
 'case': 2.386294361119891,
 'family': 2.386294361119891,
 'pair': 2.386294361119891,
 'wears': 2.386294361119891,
 'airtight': 2.386294361119891,
 'burning': 2.386294361119891,
 'two': 2.386294361119891,
 'anna': 2.386294361119891,
 'yellow': 2.386294361119891,
 'made': 2.386294361119891,
 'moon': 2.386294361119891,
 'filled': 2.386294361119891,
 'fool': 2.386294361119891,
 'na': 2.386294361119891,
 'red': 2.386294361119891,
 'shoes': 2.386294361119891,
 'hole': 2.386294361119891,
 'glass': 2.386294361119891,
 'youre': 2.386294361119891,
 'said': 2.386294361119891,
 'woman': 2.252762968495368,
 'voice': 2.386294361119891,
 'closest': 2.386294361119891,
 'white': 2.252762968495368,
 'rubber': 2.386294361119891,
 'looks': 2.386294361119891,
 'come': 2.386294361119891,
 'probably': 2.386294361119891,
 'night': 2.386294361119891,
 'far': 2.386294361119891,
 'end': 2.386294361119891,
 'sneakers': 2.3862

In [22]:
def computeTFIDF (tfBagOfWords, idfs):

  tfidf = {}

  for word, tf in tfBagOfWords.items():

    tfidf[word] = tf * idfs [word]

  return tfidf

In [23]:
import pandas as pd

In [24]:
tfidfA = computeTFIDF(tfA, idfs)

tfidfB = computeTFIDF (tfB, idfs)

df = pd.DataFrame ([tfidfA, tfidfB])

In [25]:
df

,bother,gon,annoyed,case,family,pair,wears,airtight,burning,two,...,rubber,looks,come,probably,night,far,end,sneakers,frustration,sun
0,0.000000,0.000000,0.000000,0.113633,0.113633,0.113633,0.113633,0.113633,0.113633,0.113633,...,0.113633,0.113633,0.000000,0.000000,0.000000,0.113633,0.113633,0.113633,0.000000,0.113633
1,0.132572,0.132572,0.132572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.132572,0.132572,0.132572,0.000000,0.000000,0.000000,0.132572,0.000000


In [26]:
documents = [documentA, documentB]


In [27]:
uniquewords

{'airtight',
 'anna',
 'annoyed',
 'bother',
 'burning',
 'case',
 'closest',
 'come',
 'end',
 'family',
 'far',
 'filled',
 'fool',
 'frustration',
 'glass',
 'gon',
 'hole',
 'looks',
 'made',
 'moon',
 'na',
 'night',
 'pair',
 'probably',
 'red',
 'rubber',
 'said',
 'shoes',
 'sneakers',
 'sun',
 'two',
 'voice',
 'wears',
 'white',
 'woman',
 'yellow',
 'youre'}

In [66]:
tfidf_vectorizer = TfidfVectorizer(vocabulary=uniquewords)

tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

tfidf_array = tfidf_matrix.toarray()

normalized_tfidf_array = normalize(tfidf_array, norm='l2', axis=1)


feature_names = tfidf_vectorizer.get_feature_names_out()


print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())


print("\nFeature Names (Words):")
print(feature_names)

TF-IDF Matrix:
[[0.22353702 0.         0.         0.         0.22353702 0.22353702
  0.         0.         0.22353702 0.22353702 0.22353702 0.
  0.         0.         0.22353702 0.         0.22353702 0.22353702
  0.22353702 0.         0.         0.         0.22353702 0.
  0.22353702 0.22353702 0.         0.22353702 0.22353702 0.22353702
  0.22353702 0.         0.22353702 0.15904843 0.15904843 0.22353702
  0.        ]
 [0.         0.26714212 0.26714212 0.26714212 0.         0.
  0.26714212 0.26714212 0.         0.         0.         0.26714212
  0.26714212 0.26714212 0.         0.         0.         0.
  0.         0.26714212 0.         0.26714212 0.         0.26714212
  0.         0.         0.26714212 0.         0.         0.
  0.         0.26714212 0.         0.19007382 0.19007382 0.
  0.        ]]

Feature Names (Words):
['airtight' 'anna' 'annoyed' 'bother' 'burning' 'case' 'closest' 'come'
 'end' 'family' 'far' 'filled' 'fool' 'frustration' 'glass' 'gon' 'hole'
 'looks' 'made' 'mo

In [67]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=['DocumentA', 'DocumentB'])

In [68]:
tfidf_df

,airtight,anna,annoyed,bother,burning,case,closest,come,end,family,...,shoes,sneakers,sun,two,voice,wears,white,woman,yellow,youre
DocumentA,0.223537,0.000000,0.000000,0.000000,0.223537,0.223537,0.000000,0.000000,0.223537,0.223537,...,0.223537,0.223537,0.223537,0.223537,0.000000,0.223537,0.159048,0.159048,0.223537,0.0
DocumentB,0.000000,0.267142,0.267142,0.267142,0.000000,0.000000,0.267142,0.267142,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.267142,0.000000,0.190074,0.190074,0.000000,0.0


In [31]:
df

,bother,gon,annoyed,case,family,pair,wears,airtight,burning,two,...,rubber,looks,come,probably,night,far,end,sneakers,frustration,sun
0,0.000000,0.000000,0.000000,0.113633,0.113633,0.113633,0.113633,0.113633,0.113633,0.113633,...,0.113633,0.113633,0.000000,0.000000,0.000000,0.113633,0.113633,0.113633,0.000000,0.113633
1,0.132572,0.132572,0.132572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.132572,0.132572,0.132572,0.000000,0.000000,0.000000,0.132572,0.000000
